# Mujoco demo

## Get mujoco robot assets

In [ ]:
!git clone https://github.com/google-deepmind/mujoco_menagerie.git

In [ ]:
!pip install mujoco

In [ ]:
from utils import display_desktop
display_desktop()

## List all robots

In [ ]:
!ls mujoco_menagerie

## Load robot in Mujoco

In [ ]:
import os
import time
import mujoco
import mujoco.viewer


robot = "universal_robots_ur10e"
# Path to MJCF XML file
XML_PATH = f"{os.getcwd()}/mujoco_menagerie/{robot}/scene.xml"

m = mujoco.MjModel.from_xml_path(XML_PATH)
d = mujoco.MjData(m)

with mujoco.viewer.launch_passive(m, d) as viewer:
    # Rendering flags
    viewer.user_scn.flags[mujoco.mjtRndFlag.mjRND_SHADOW] = 0
    viewer.user_scn.flags[mujoco.mjtRndFlag.mjRND_REFLECTION] = 0
    viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT]
    
    # Close the viewer automatically after 30 wall-seconds.
    start = time.time()
    while viewer.is_running() and time.time() - start < 30:
        step_start = time.time()
        
        # mj_step can be replaced with code that also evaluates
        # a policy and applies a control signal before stepping the physics.
        mujoco.mj_step(m, d)
        
        # Pick up changes to the physics state, apply perturbations, update options from GUI.
        viewer.sync()
        
        # Rudimentary time keeping, will drift relative to wall clock.
        time_until_next_step = m.opt.timestep - (time.time() - step_start)
        if time_until_next_step > 0:
            time.sleep(time_until_next_step)